# Capstone Final Assignment

##### IMPORT  

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_data_1 = pd.read_csv(body)
df_data_1.rename(columns={'744301': 'Postal Code'}, inplace=True)
df_data_1.rename(columns={'9.1167': 'lat'}, inplace=True)
df_data_1.rename(columns={'92.8': 'lng'}, inplace=True)
df_data_1.head()

,IN,Postal Code,Kakana,Andaman & Nicobar Islands,01,Nicobar,638,Carnicobar,Unnamed: 8,lat,lng,4
0,IN,744301,Sawai,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,7.5166,93.6031,4
1,IN,744301,Lapathy,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3
2,IN,744301,Mus,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.2333,92.7833,4
3,IN,744301,Carnicobar,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3
4,IN,744302,Campbelbay,Andaman & Nicobar Islands,1,Nicobar,638.0,Nancowrie,NaN,9.1833,92.7667,1


In [2]:
ba=df_data_1[['Postal Code','lat','lng']]

In [3]:
ba.drop_duplicates(subset ="Postal Code", 
                     keep = False, inplace = True)
ba.shape

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(1877, 3)

##### WEB Scrapping 

In [4]:
import urllib.request
url = "https://www.indiatvnews.com/pincode/karnataka/bangalore/bangalore-city"
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify())

In [5]:
all_tables=soup.find("div", attrs={"class": "lhs mb30"})
right_table=all_tables.find("table", attrs={"class": "alt"})
lst=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==5:
        lst.append([cells[0].text.rstrip(),cells[1].text.rstrip(),cells[2].text.rstrip(),cells[3].text.rstrip(),cells[4].text.rstrip()])

In [47]:
import pandas as pd
cols = ['Neighborhood', 'Div', 'City', 'State', 'Postal Code']
df = pd.DataFrame(lst, columns=cols)
df = df.drop(['City','State','Div'],axis=1)
df=df.astype({'Postal Code': 'int64'})
d = pd.merge(df,ba, on="Postal Code",how='inner')
d.to_csv('banglore_data.csv')

In [48]:
df = pd.read_csv('banglore_data.csv', index_col=0)
df.head()

,Neighborhood,Postal Code,lat,lng
0,Bangalore International Airport,560300,13.0082,77.5293
1,Benson Town,560046,13.2257,77.5750
2,C.V.Raman Nagar,560093,13.1077,77.5810
3,Domlur,560071,13.2257,77.5750
4,Fraser Town,560005,12.9910,77.5843


In [49]:
locations = df[['lat','lng']]
locationlist = locations.values.tolist()

In [50]:
import pandas as pd
import requests
from xml.etree import ElementTree
import numpy as np
!pip -q install folium
import folium

In [51]:
lat=12.9716
long=77.5946
map = folium.Map(location=[12.9716,77.5946], zoom_start=10)
for lat, lng, label in zip(df['lat'], df['lng'], df['Postal Code']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)
map

In [52]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LBTVKAUHJ02FZAJ4PXSX1MFFNU2NK0QZ0FFNU4UDJOJLZVJS
CLIENT_SECRET:WV1N2KMTI3ELNLSINDDJ4PNDVBMLXNGVLVNGF3HDPJCILOW0


In [53]:
df.loc[0, 'Neighborhood']

'Bangalore International Airport'

In [54]:
neighborhood_latitude = df.loc[0, 'lat'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'lng'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bangalore International Airport are 13.0082, 77.5293.


In [55]:
LIMIT=100
radius =5000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LBTVKAUHJ02FZAJ4PXSX1MFFNU2NK0QZ0FFNU4UDJOJLZVJS&client_secret=WV1N2KMTI3ELNLSINDDJ4PNDVBMLXNGVLVNGF3HDPJCILOW0&v=20180605&ll=13.0082,77.5293&radius=5000&limit=100'

In [56]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [57]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,"PVR Gold Class, Orion",Multiplex,13.010923,77.555010
1,Orion Mall,Shopping Mall,13.011313,77.555039
2,Natural Ice Cream,Ice Cream Shop,12.985484,77.543158
3,PVR bluO,Bowling Alley,13.011198,77.555057
4,O.G. Variar & Sons Bakery,Bakery,12.987669,77.549978


In [58]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
bang_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['lat'],
                                   longitudes=df['lng']
                                  )

Bangalore International Airport
Benson Town
C.V.Raman Nagar
Domlur
Fraser Town
G.K.V.K.
H.K.P. Road
Indiranagar Bangalore
Maruthi Sevanagar
Sadashivanagar
Sivan Chetty Gardens
Adugodi
Anjanapura
Carmelram
Chandapura
Chickpet
Electronics City
Wipro Limited
HSR Layout
Koramangala VI Bk
Rv Niketan
Thalaghattapura
Ullalu Upanagar
Hampinagar
Jalahalli West
Kodigehalli
Magadi Road
Malleswaram West
Nandinilayout
Peenya Dasarahalli
Science Institute
Seshadripuram
Vidyaranyapura


In [62]:
print(bang_venues.shape)
bang_venues.head()

(1298, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bangalore International Airport,13.0082,77.5293,"PVR Gold Class, Orion",13.010923,77.555010,Multiplex
1,Bangalore International Airport,13.0082,77.5293,Orion Mall,13.011313,77.555039,Shopping Mall
2,Bangalore International Airport,13.0082,77.5293,Natural Ice Cream,12.985484,77.543158,Ice Cream Shop
3,Bangalore International Airport,13.0082,77.5293,PVR bluO,13.011198,77.555057,Bowling Alley
4,Bangalore International Airport,13.0082,77.5293,O.G. Variar & Sons Bakery,12.987669,77.549978,Bakery


In [63]:
bang_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adugodi,4,4,4,4,4,4
Anjanapura,4,4,4,4,4,4
Bangalore International Airport,100,100,100,100,100,100
Benson Town,4,4,4,4,4,4
C.V.Raman Nagar,33,33,33,33,33,33
Carmelram,66,66,66,66,66,66
Chandapura,4,4,4,4,4,4
Chickpet,100,100,100,100,100,100
Domlur,4,4,4,4,4,4


In [64]:
print('There are {} uniques categories.'.format(len(bang_venues['Venue Category'].unique())))

There are 107 uniques categories.


In [65]:
bang_venues['Venue Category'].unique()[:30]

array(['Multiplex', 'Shopping Mall', 'Ice Cream Shop', 'Bowling Alley',
       'Bakery', 'Lounge', 'Italian Restaurant', 'Department Store',
       'Hotel', 'Gym / Fitness Center', 'Clothing Store', 'Donut Shop',
       'Movie Theater', 'Pub', 'Indian Restaurant',
       'Fast Food Restaurant', 'French Restaurant', 'Breakfast Spot',
       'Coffee Shop', 'Snack Place', 'Electronics Store',
       'Mexican Restaurant', 'Bar', 'Motorcycle Shop', 'Asian Restaurant',
       'South Indian Restaurant', 'Vegetarian / Vegan Restaurant',
       'Seafood Restaurant', 'Restaurant', 'Gas Station'], dtype=object)

In [66]:
# one hot encoding
bang_onehot = pd.get_dummies(bang_venues[['Venue Category']], prefix="", prefix_sep="")
# add postal, borough and neighborhood column back to dataframe
bang_onehot['Neighborhoods'] = bang_venues['Neighborhood'] 
# move postal, borough and neighborhood column to the first column
fixed_columns = list(bang_onehot.columns[-1:]) + list(bang_onehot.columns[:-1])
bang_onehot = bang_onehot[fixed_columns]
print(bang_onehot.shape)
bang_onehot.head(10)

(1298, 108)


,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,...,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant
0,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bangalore International Airport,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Bangalore International Airport,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Bangalore International Airport,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
bang_grouped = bang_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(bang_grouped.shape)
bang_grouped.head(20)

(33, 108)


,Neighborhoods,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,...,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant
0,Adugodi,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
1,Anjanapura,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
2,Bangalore International Airport,0.00,0.000000,0.000000,0.00,0.01,0.00,0.010000,0.000000,0.040000,...,0.000000,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.000000,0.030000
3,Benson Town,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
4,C.V.Raman Nagar,0.00,0.030303,0.030303,0.00,0.00,0.00,0.000000,0.000000,0.030303,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.030303,0.000000
5,Carmelram,0.00,0.000000,0.000000,0.00,0.00,0.00,0.015152,0.015152,0.030303,...,0.000000,0.00,0.015152,0.00,0.00,0.00,0.00,0.00,0.000000,0.015152
6,Chandapura,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
7,Chickpet,0.01,0.000000,0.010000,0.01,0.00,0.01,0.010000,0.000000,0.030000,...,0.000000,0.01,0.000000,0.01,0.02,0.00,0.01,0.01,0.000000,0.000000
8,Domlur,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
9,Electronics City,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000


In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bang_grouped['Neighborhoods']

for ind in np.arange(bang_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bang_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adugodi,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
1,Anjanapura,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
2,Bangalore International Airport,Indian Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Multiplex,Fast Food Restaurant,Clothing Store,Hotel,Donut Shop,Bakery
3,Benson Town,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
4,C.V.Raman Nagar,Café,Indian Restaurant,Chinese Restaurant,Ice Cream Shop,Pizza Place,Smoke Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Bus Station


### K-Means Clustering

In [73]:
kclusters = 5
from sklearn.cluster import KMeans

bang_grouped_clustering = bang_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bang_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]

array([0, 0, 1, 0, 4, 1, 0, 1, 0, 0, 1, 1, 1, 4, 4, 0, 1, 3, 4, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 2, 4, 4, 0, 0], dtype=int32)

In [74]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [75]:
bang_merged =df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bang_merged = bang_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
bang_merged

,Neighborhood,Postal Code,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bangalore International Airport,560300,13.0082,77.5293,1,Indian Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Multiplex,Fast Food Restaurant,Clothing Store,Hotel,Donut Shop,Bakery
1,Benson Town,560046,13.2257,77.5750,0,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
2,C.V.Raman Nagar,560093,13.1077,77.5810,4,Café,Indian Restaurant,Chinese Restaurant,Ice Cream Shop,Pizza Place,Smoke Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Bus Station
3,Domlur,560071,13.2257,77.5750,0,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
4,Fraser Town,560005,12.9910,77.5843,1,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Bakery,Italian Restaurant,Seafood Restaurant,Shopping Mall,Department Store
5,G.K.V.K.,560065,12.9881,77.5052,1,Indian Restaurant,Ice Cream Shop,Café,Pizza Place,Seafood Restaurant,Gym,Fast Food Restaurant,Bakery,Breakfast Spot,Miscellaneous Shop
6,H.K.P. Road,560051,12.9979,77.5861,1,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Burger Joint,Department Store,Italian Restaurant,Seafood Restaurant,Bakery
7,Indiranagar Bangalore,560038,13.2257,77.5750,0,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
8,Maruthi Sevanagar,560033,13.0108,77.7494,1,Hotel,Indian Restaurant,Café,Pizza Place,Restaurant,Ice Cream Shop,Coffee Shop,Eastern European Restaurant,Bakery,Department Store
9,Sadashivanagar,560080,13.2257,77.5750,0,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop


In [76]:
map_clusters = folium.Map(location=[12.9716,77.5946], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bang_merged['lat'],bang_merged['lng'],bang_merged['Neighborhood'],bang_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [77]:
bang_merged.loc[bang_merged['Cluster Labels'] == 0,bang_merged.columns[[1] + list(range(5,bang_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,560046,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
3,560071,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
7,560038,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
9,560080,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
11,560030,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
12,560108,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
14,560081,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
16,560100,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
17,560100,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop
26,560023,Resort,Cupcake Shop,Light Rail Station,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Donut Shop


#### Cluster 2

In [78]:
bang_merged.loc[bang_merged['Cluster Labels'] == 1,bang_merged.columns[[1] + list(range(5,bang_merged.shape[1]))]]


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560300,Indian Restaurant,Ice Cream Shop,Coffee Shop,Italian Restaurant,Multiplex,Fast Food Restaurant,Clothing Store,Hotel,Donut Shop,Bakery
4,560005,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Bakery,Italian Restaurant,Seafood Restaurant,Shopping Mall,Department Store
5,560065,Indian Restaurant,Ice Cream Shop,Café,Pizza Place,Seafood Restaurant,Gym,Fast Food Restaurant,Bakery,Breakfast Spot,Miscellaneous Shop
6,560051,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Burger Joint,Department Store,Italian Restaurant,Seafood Restaurant,Bakery
8,560033,Hotel,Indian Restaurant,Café,Pizza Place,Restaurant,Ice Cream Shop,Coffee Shop,Eastern European Restaurant,Bakery,Department Store
13,560035,Hotel,Indian Restaurant,Café,Pizza Place,Restaurant,Ice Cream Shop,Coffee Shop,Eastern European Restaurant,Bakery,Department Store
15,560053,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Burger Joint,Department Store,Italian Restaurant,Seafood Restaurant,Bakery
20,560059,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Burger Joint,Department Store,Italian Restaurant,Seafood Restaurant,Bakery
24,560015,Indian Restaurant,Ice Cream Shop,Bakery,Hotel,Lounge,Pub,Snack Place,Brewery,Sandwich Place,Coffee Shop
27,560055,Indian Restaurant,Hotel,Ice Cream Shop,Pub,Lounge,Burger Joint,Department Store,Italian Restaurant,Seafood Restaurant,Bakery


#### Cluster 3

In [79]:
bang_merged.loc[bang_merged['Cluster Labels'] == 2,bang_merged.columns[[1] + list(range(5,bang_merged.shape[1]))]]


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,560042,Mountain,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Donut Shop,Eastern European Restaurant


#### Cluster 4

In [80]:
bang_merged.loc[bang_merged['Cluster Labels'] == 3,bang_merged.columns[[1] + list(range(5,bang_merged.shape[1]))]]


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,560112,Café,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Flea Market,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Donut Shop


#### Cluster 5

In [82]:
bang_merged.loc[bang_merged['Cluster Labels'] ==4 ,bang_merged.columns[[1] + list(range(5,bang_merged.shape[1]))]]


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,560093,Café,Indian Restaurant,Chinese Restaurant,Ice Cream Shop,Pizza Place,Smoke Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Bus Station
18,560102,Café,Fast Food Restaurant,Food Truck,Grocery Store,Pizza Place,Vegetarian / Vegan Restaurant,Sandwich Place,Brewery,Burger Joint,Coffee Shop
19,560095,Café,Indian Restaurant,Chinese Restaurant,Ice Cream Shop,Pizza Place,Smoke Shop,Coffee Shop,Clothing Store,Fast Food Restaurant,Bus Station
21,560109,Café,Fast Food Restaurant,Food Truck,Grocery Store,Pizza Place,Vegetarian / Vegan Restaurant,Sandwich Place,Brewery,Burger Joint,Coffee Shop
22,560110,Café,Indian Restaurant,Hotel,Seafood Restaurant,Smoke Shop,Dessert Shop,South Indian Restaurant,Bakery,Shopping Mall,Clothing Store
23,560104,Café,Fast Food Restaurant,Food Truck,Grocery Store,Pizza Place,Vegetarian / Vegan Restaurant,Sandwich Place,Brewery,Burger Joint,Coffee Shop
